In [104]:
import pandas as pd
import numpy as np
import sys
import os
from unidecode import unidecode


In [105]:
DEBUG = False

# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../") if DEBUG else os.path.abspath("../../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [106]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

In [107]:
def processar_disciplina(disciplina):
    """
    Processa uma string de detalhes de disciplina, dividindo em componentes.

    Args:
        disciplina (str): String contendo os detalhes da disciplina.

    Returns:
        tuple: Uma tupla contendo os detalhes processados da disciplina.
    """
    detalhes_disciplinas = disciplina.split(" - ")
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = " - ".join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)


def separar_disciplina_e_codigo(x):
    """
    Extrai código e nome da disciplina a partir de uma lista de strings.

    Args:
        x (list): Lista contendo os detalhes da disciplina.

    Returns:
        tuple: Uma tupla com o código e o nome da disciplina, ou NaN se não disponível.
    """
    if x[0] == "nan":
        return np.nan
    try:
        return (
            x[0],
            x[1].split()[0].replace(" ", ""),
            " ".join(x[1].split()[1:]).strip(),
            x[2].strip(),
            x[3].strip()
        )
    except IndexError:
        print("Erro de índice para:", x)
        return None
    

In [108]:
# Leitura do arquivo
df_silver = extrair_dados_database("alunos_ufrj.parquet", "silver")

# Carregando as dimensoes
df_d_aluno = extrair_dados_database("d_aluno.parquet", "gold")
df_d_periodo = extrair_dados_database("d_periodo.parquet", "gold")
df_d_curso = extrair_dados_database("d_curso.parquet", "gold")
df_d_periodo = extrair_dados_database("d_periodo.parquet", "gold")
df_d_disciplina = extrair_dados_database("d_disciplina.parquet", "gold")
df_d_situacao = extrair_dados_database("d_situacao.parquet", "gold")

In [109]:
# Remove as linhas onde a coluna 'disciplinasCursadas' tem NaN ou None
df_silver = df_silver.dropna(subset=["disciplinasCursadas"])

In [110]:
# Processa a coluna 'disciplinasCursadas' para separar em periodo, disciplina, grau, situação
df_silver["disciplinasCursadas"] = df_silver["disciplinasCursadas"].apply(
    lambda row: [processar_disciplina(item) for item in row.split("\n")]
)

In [111]:
# Explode a coluna 'disciplinasCursadas' para dividir as listas em linhas separadas
df_exploded = df_silver.explode("disciplinasCursadas")

In [112]:
# Processa os detalhes das disciplinas
df_exploded["disciplinasCursadas"] = df_exploded["disciplinasCursadas"].apply(
    separar_disciplina_e_codigo
)

In [113]:
# Descompacta as tuplas em colunas separadas
df_exploded[
    [
        "periodo",
        "codDisciplina",
        "nomeDisciplina",
        "grauDisciplina",
        "situacaoDisciplina",
    ]
] = pd.DataFrame(df_exploded["disciplinasCursadas"].tolist(), index=df_exploded.index)

In [114]:
def ajustar_grau_disciplina(grau):
    if pd.isna(grau) or grau.strip() == '':
        grau = 'NCG' # Não Confere Grau

    elif grau.isdigit():
        grau = str(float(grau)/10)

    return grau

# Aplicando a função ao DataFrame
df_exploded['grauDisciplina'] = df_exploded['grauDisciplina'].apply(ajustar_grau_disciplina)

In [115]:
# Remover acentos e cedilhas
df_exploded['nomeDisciplina'] = df_exploded['nomeDisciplina'].apply(unidecode)

# Dicionário de substituições de disciplinas
dicionario_substituicoes = {
    'Top Esp em Eng de Softaware': 'Top Esp em Eng de Software',
    'Fisico-quimica I': 'Fisico-Quimica I',
    'Topicos Especiais em Redes': 'Topicos Especiais em Redes I',
    # Adicione mais substituições conforme necessário
}

# Função para substituir frases com base no dicionário
def substituir_disciplina(frase):
    return dicionario_substituicoes.get(frase, frase)

df_exploded['nomeDisciplina'] = df_exploded['nomeDisciplina'].apply(substituir_disciplina)

In [116]:
desempenho_academico = df_exploded[['periodo', 'codCursoIngresso', 'codCursoAtual', 'matriculaDre', 'codDisciplina', 'nomeDisciplina', 'grauDisciplina', 'situacaoDisciplina']]

In [117]:
# Padronizando nome das colunas
desempenho_academico = desempenho_academico.rename(
    columns={
        "periodo": "DS_PERIODO",
        "matriculaDre": "DS_MATRICULA_DRE",
        "codDisciplina": "CD_DISCIPLINA",
        "nomeDisciplina": "DS_NOME_DISCIPLINA",
        "grauDisciplina": "VL_GRAU_DISCIPLINA",
        "situacaoDisciplina": "DS_SITUACAO_DETALHADA",
        "codCursoIngresso": "CD_CURSO_INGRESSO",
        "codCursoAtual": "CD_CURSO_ATUAL",
    }
)

In [120]:
f_desempenho_academico = pd.merge(desempenho_academico, df_d_aluno, on='DS_MATRICULA_DRE', how='inner')
f_desempenho_academico = pd.merge(f_desempenho_academico, df_d_disciplina, on=['CD_DISCIPLINA','DS_NOME_DISCIPLINA'], how='inner')
f_desempenho_academico = pd.merge(f_desempenho_academico, df_d_periodo, on='DS_PERIODO', how='inner')
f_desempenho_academico = pd.merge(f_desempenho_academico, df_d_curso, on=['CD_CURSO_INGRESSO', 'CD_CURSO_ATUAL'], how='inner')
f_desempenho_academico = pd.merge(f_desempenho_academico, df_d_situacao, on=['DS_SITUACAO_DETALHADA'], how='inner')

In [122]:
f_desempenho_academico = f_desempenho_academico[['SK_D_ALUNO', 'SK_D_DISCIPLINA', 'SK_D_CURSO', 'SK_D_PERIODO', 'SK_D_SITUACAO', 'VL_GRAU_DISCIPLINA']]

In [281]:
# Salvar dados na camada Gold
carregar_dados_database(f_desempenho_academico, "f_desempenho_academico.parquet", "gold")